# 00-Get-Data

Just find the data and download it.

## System

In [ ]:
cd ..

In [ ]:
!pwd

## Imports 

In [ ]:
# built in modules
import os
import json

In [ ]:
# third party modules
import pandas as pd

In [ ]:
# custom modules
from gbs.etl.urls import Urls
from gbs.helpers import runcmd
from gbs.etl.extract import Extract

## Build Data Directories

In [ ]:
# make data/ directory if it doesn't exist
if not os.path.exists("data"):
    os.makedirs("data")

In [ ]:
# make data/source/ directory if it doesn't exist
if not os.path.exists("data/source"):
    os.makedirs("data/source/")

## Download Data

In [ ]:
# download crops
runcmd("wget -O data/source/crops.csv " + Urls.crops_url)

In [ ]:
# download countries specs
runcmd("wget -O data/source/country_specs_url.csv " + Urls.country_specs_url)

In [ ]:
# dowlnoad qcl metadata
runcmd("wget -O data/source/production.json " + Urls.production_json)

## Manage Production

In [ ]:
# load qcl metadata
qcl = json.load(open("data/source/production.json"))
qcl

In [ ]:
# file qcl json is a list of datasets
dataset = qcl.get("Datasets").get("Dataset")
dataset[:3]

In [ ]:
# convert to pandas dataframe
df = pd.DataFrame(dataset)
df.head()

In [ ]:
# find qcl url
production_url = df.loc[df.DatasetCode == "QCL"].FileLocation.values[0]
production_url

In [ ]:
# health check
assert production_url == Urls.production_url

In [ ]:
# curl qlc.zip
runcmd(f"curl --output ./data/source/production.zip '{Urls.production_url}'")

In [ ]:
# unzip the file
runcmd(f"unzip ./data/source/production.zip -d ./data/source/")

In [ ]:
# make data/source/production if it doesn't exist
if not os.path.exists("data/source/production"):
    os.makedirs("data/source/production")

In [ ]:
# move production files to production folder
runcmd("mv ./data/source/Pr* ./data/source/production/")

In [ ]:
# update filenames to lowercase
path = "data/source/production/"
pattern = "Production_Crops_Livestock_E_All_"

for fn in os.listdir(path):
    # clean filename
    cleaned_fn = (
        fn.replace(pattern, "")
        .lower()
        .replace("(", "")
        .replace(")", "")
    )

    # src and dst
    src_ = os.path.join(path, fn)
    dst_ = os.path.join(path, cleaned_fn)

    # do rename
    os.rename(src_, dst_)


In [ ]:
os.listdir("data/source/production/")

In [ ]:
!rm -rf data/source/production.zip

In [ ]:
!rm -rf data/source/production.json

## Check Data Integrity

In [ ]:
ext  = ".csv"
path = "data/source"

for fn in os.listdir(path):

    print(fn)

    if not fn.endswith(ext):
        continue

    df = pd.read_csv(os.path.join(path, fn))
    assert isinstance(df, pd.DataFrame)
    assert df.shape[0] > 0
    assert df.shape[1] > 0

In [ ]:
ext = ".csv"
path = "data/source/production/"

for fn in os.listdir(path):
    print(fn)

    if not fn.endswith(ext):
        continue

    df = pd.read_csv(os.path.join(path, fn))
    assert isinstance(df, pd.DataFrame)
    assert df.shape[0] > 0
    assert df.shape[1] > 0

## ... or Just use relevant classes

### One by One

In [ ]:
extract = Extract()

In [ ]:
extract.__dict__

In [ ]:
extract.clean()

In [ ]:
extract.make_folders()

In [ ]:
extract.get_crops()

In [ ]:
extract.get_country_specs()

In [ ]:
extract.get_production()

In [ ]:
extract.check_files()

### All at Once

In [ ]:
extract = Extract()
extract.get_all(clean=True)